In [20]:
from tensorflow.keras.applications.resnet50 import ResNet50,decode_predictions,preprocess_input
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten
import numpy as np
# plt.style.use('ggplot')

# 전이학습으로 학습한 후 test(cured) 1번째 이미지 분류

In [21]:
# imgData = image.load_img('dental_image/train/cured/301.jpg')
# imgData

In [22]:
dataGen = image.ImageDataGenerator(rescale=1./255 )
trainGen = dataGen.flow_from_directory('dental_image/train' ,target_size=(98,98),batch_size=210)
x_train,y_train = trainGen.next()

Found 150 images belonging to 3 classes.


In [23]:
# 분류값 확인
trainGen.class_indices

{'cured': 0, 'decayed': 1, 'healthy': 2}

In [24]:
x_train.shape, y_train.shape

((150, 98, 98, 3), (150, 3))

In [25]:
model_conv  = ResNet50( weights='imagenet',include_top=False, input_shape=(98,98,3))

In [26]:
# 트징 추출기는 학습시 제외
for layer in model_conv.layers:
    layer.trainable = False

In [27]:
model_conv.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 98, 98, 3)]  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 104, 104, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 49, 49, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 49, 49, 64)   256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [28]:
model = Sequential()
model.add(model_conv) # con, relu, pool     + dropout은 아무때나
model.add(Flatten())
model.add(Dense(units=64,activation='relu'))
model.add(Dense(units=3,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(x_train,y_train,epochs=100)

Epoch 1/100
5/5 [==============================] - 1s 175ms/step - loss: 5.5964 - acc: 0.3933
Epoch 2/100
5/5 [==============================] - 1s 176ms/step - loss: 1.7974 - acc: 0.3400
Epoch 3/100
5/5 [==============================] - 1s 173ms/step - loss: 1.3324 - acc: 0.3267
Epoch 4/100
5/5 [==============================] - 1s 170ms/step - loss: 1.1529 - acc: 0.4400
Epoch 5/100
5/5 [==============================] - 1s 173ms/step - loss: 1.0987 - acc: 0.4067
Epoch 6/100
5/5 [==============================] - 1s 180ms/step - loss: 1.1237 - acc: 0.3867
Epoch 7/100
5/5 [==============================] - 1s 185ms/step - loss: 1.0535 - acc: 0.3933
Epoch 8/100
5/5 [==============================] - 1s 174ms/step - loss: 1.0404 - acc: 0.5333
Epoch 9/100
5/5 [==============================] - 1s 180ms/step - loss: 1.0736 - acc: 0.3200
Epoch 10/100
5/5 [==============================] - 1s 184ms/step - loss: 1.0178 - acc: 0.4867
Epoch 11/100
5/5 [==============================] - 1s 185m

5/5 [==============================] - 1s 164ms/step - loss: 0.7898 - acc: 0.6600
Epoch 88/100
5/5 [==============================] - 1s 165ms/step - loss: 0.8636 - acc: 0.5333
Epoch 89/100
5/5 [==============================] - 1s 165ms/step - loss: 0.7963 - acc: 0.6400
Epoch 90/100
5/5 [==============================] - 1s 171ms/step - loss: 0.8609 - acc: 0.5333
Epoch 91/100
5/5 [==============================] - 1s 166ms/step - loss: 0.8808 - acc: 0.6400
Epoch 92/100
5/5 [==============================] - 1s 176ms/step - loss: 0.9180 - acc: 0.5200
Epoch 93/100
5/5 [==============================] - 1s 177ms/step - loss: 0.8477 - acc: 0.6133
Epoch 94/100
5/5 [==============================] - 1s 162ms/step - loss: 0.7669 - acc: 0.6800
Epoch 95/100
5/5 [==============================] - 1s 158ms/step - loss: 0.7570 - acc: 0.6733
Epoch 96/100
5/5 [==============================] - 1s 166ms/step - loss: 0.7841 - acc: 0.6533
Epoch 97/100
5/5 [==============================] - 1s 157ms/st

In [29]:
testImg = image.load_img('dental_image/test/cured/301.jpg',target_size=(98,98))


In [30]:
type(testImg)

PIL.Image.Image

In [31]:
imgArr = image.img_to_array(testImg) #  넘파이 배열로 만듦
imgArr = imgArr/255   #  스케일링

In [32]:
model.predict(imgArr.reshape(1,98,98,3)).argmax(axis=1)

array([0], dtype=int64)

In [33]:
trainGen.class_indices

{'cured': 0, 'decayed': 1, 'healthy': 2}

In [34]:
model.evaluate(x_train,y_train)

5/5 [==============================] - 1s 175ms/step - loss: 0.7239 - acc: 0.6733


[0.7238516211509705, 0.6733333468437195]

In [35]:
testGen = dataGen.flow_from_directory( 'dental_image/test', target_size=(98,98),batch_size=60)
x_test , y_test = testGen.next()

Found 60 images belonging to 3 classes.


In [36]:
list(trainGen.class_indices.keys())[list(model.predict(preprocess_input(np.expand_dims(imgArr, axis=0))).argmax(axis=1))[0]]

'decayed'

In [37]:
model.evaluate(x_test,y_test)

2/2 [==============================] - 0s 97ms/step - loss: 0.8800 - acc: 0.5833


[0.8799863457679749, 0.5833333134651184]

In [38]:
model.predict(preprocess_input(np.expand_dims(imgArr,axis=0))).argmax(axis=1)[0]

0